# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [3]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [4]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [5]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [6]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
print_every = 100
print(f"Using device: {device}")

Using device: /GPU:0


In [8]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [9]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu', kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu', kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [10]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [11]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [12]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.8376107215881348, Accuracy: 12.5, Val Loss: 2.972851276397705, Val Accuracy: 14.399999618530273
Iteration 100, Epoch 1, Loss: 2.217054843902588, Accuracy: 28.511756896972656, Val Loss: 1.8708001375198364, Val Accuracy: 37.599998474121094
Iteration 200, Epoch 1, Loss: 2.0683774948120117, Accuracy: 32.28389358520508, Val Loss: 1.8711544275283813, Val Accuracy: 39.0
Iteration 300, Epoch 1, Loss: 1.99090576171875, Accuracy: 34.276371002197266, Val Loss: 1.8913390636444092, Val Accuracy: 37.099998474121094
Iteration 400, Epoch 1, Loss: 1.921471357345581, Accuracy: 36.07387924194336, Val Loss: 1.736506700515747, Val Accuracy: 42.20000076293945
Iteration 500, Epoch 1, Loss: 1.8781315088272095, Accuracy: 37.14446258544922, Val Loss: 1.6567838191986084, Val Accuracy: 42.89999771118164
Iteration 600, Epoch 1, Loss: 1.8487906455993652, Accuracy: 38.0667610168457, Val Loss: 1.6726183891296387, Val Accuracy: 42.39999771118164
Iteration 700, Epoch 1, Loss: 1.82432711124

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [13]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.6221108436584473, Accuracy: 7.8125, Val Loss: 5.1392669677734375, Val Accuracy: 11.300000190734863
Iteration 100, Epoch 1, Loss: 2.0717313289642334, Accuracy: 28.728342056274414, Val Loss: 1.7378164529800415, Val Accuracy: 39.0
Iteration 200, Epoch 1, Loss: 1.870407223701477, Accuracy: 34.895835876464844, Val Loss: 1.5448405742645264, Val Accuracy: 46.79999923706055
Iteration 300, Epoch 1, Loss: 1.7649301290512085, Accuracy: 38.263084411621094, Val Loss: 1.4643347263336182, Val Accuracy: 48.5
Iteration 400, Epoch 1, Loss: 1.6759995222091675, Accuracy: 41.13154602050781, Val Loss: 1.3651281595230103, Val Accuracy: 51.30000305175781
Iteration 500, Epoch 1, Loss: 1.616948127746582, Accuracy: 43.07322692871094, Val Loss: 1.3385525941848755, Val Accuracy: 53.500003814697266
Iteration 600, Epoch 1, Loss: 1.5747392177581787, Accuracy: 44.594947814941406, Val Loss: 1.2964648008346558, Val Accuracy: 54.29999923706055
Iteration 700, Epoch 1, Loss: 1.5383704900741577

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [14]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 0, Epoch 1, Loss: 3.074888229370117, Accuracy: 9.375, Val Loss: 2.82431960105896, Val Accuracy: 11.800000190734863
Iteration 100, Epoch 1, Loss: 2.1951799392700195, Accuracy: 28.94492530822754, Val Loss: 1.8773443698883057, Val Accuracy: 36.79999923706055
Iteration 200, Epoch 1, Loss: 2.0503733158111572, Accuracy: 32.71144485473633, Val Loss: 1.8234233856201172, Val Accuracy: 39.79999923706055
Iteration 300, Epoch 1, Loss: 1.9781956672668457, Accuracy: 34.61378860473633, Val Loss: 1.8512126207351685, Val Accuracy: 37.79999923706055
Iteration 400, Epoch 1, Loss: 1.9135711193084717, Accuracy: 36.45573425292969, Val Loss: 1.6924000978469849, Val Accuracy: 43.79999923706055
Iteration 500, Epoch 1, Loss: 1.872481346130371, Accuracy: 37.49064636230469, Val Loss: 1.6183549165725708, Val Accuracy: 43.70000076293945
Iteration 600, Epoch 1, Loss: 1.8436527252197266, Accuracy: 38.33454513549805, Val Loss: 1.661826252937317, Val Accuracy: 44.20000076293945
Iteration 700, Epoch 1, Loss: 1

Альтернативный менее гибкий способ обучения:

In [15]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2.0416 - sparse_categorical_accuracy: 0.3346 - val_loss: 1.5892 - val_sparse_categorical_accuracy: 0.4440
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.5796 - sparse_categorical_accuracy: 0.4550


[1.5969064235687256, 0.4499000012874603]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [16]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (32, 32, 3)
    channel_1, channel_2, num_classes = 32, 16, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu', kernel_initializer=initializer, input_shape=input_shape),
        tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 0, Epoch 1, Loss: 2.9829769134521484, Accuracy: 18.75, Val Loss: 2.7378580570220947, Val Accuracy: 12.700000762939453
Iteration 100, Epoch 1, Loss: 1.9940592050552368, Accuracy: 30.151609420776367, Val Loss: 1.7747437953948975, Val Accuracy: 37.79999923706055
Iteration 200, Epoch 1, Loss: 1.8612366914749146, Accuracy: 35.02021026611328, Val Loss: 1.641056776046753, Val Accuracy: 45.0
Iteration 300, Epoch 1, Loss: 1.789518117904663, Accuracy: 37.21968460083008, Val Loss: 1.6286861896514893, Val Accuracy: 44.60000228881836
Iteration 400, Epoch 1, Loss: 1.7298997640609741, Accuracy: 39.23784255981445, Val Loss: 1.5590379238128662, Val Accuracy: 47.20000076293945
Iteration 500, Epoch 1, Loss: 1.6867098808288574, Accuracy: 40.637474060058594, Val Loss: 1.518873929977417, Val Accuracy: 46.39999771118164
Iteration 600, Epoch 1, Loss: 1.659196138381958, Accuracy: 41.63373565673828, Val Loss: 1.4772602319717407, Val Accuracy: 49.900001525878906
Iteration 700, Epoch 1, Loss: 1.63404572

In [17]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 1.8439 - sparse_categorical_accuracy: 0.3734 - val_loss: 1.3699 - val_sparse_categorical_accuracy: 0.5000
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.3374 - sparse_categorical_accuracy: 0.5265


[1.3441364765167236, 0.5184000134468079]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [18]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [19]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0859813690185547, Accuracy: 9.375, Val Loss: 2.776162624359131, Val Accuracy: 13.500000953674316
Iteration 100, Epoch 1, Loss: 2.220487594604492, Accuracy: 28.898515701293945, Val Loss: 1.8953557014465332, Val Accuracy: 38.5
Iteration 200, Epoch 1, Loss: 2.0606679916381836, Accuracy: 32.57929229736328, Val Loss: 1.794450044631958, Val Accuracy: 41.10000228881836
Iteration 300, Epoch 1, Loss: 1.9926700592041016, Accuracy: 34.276371002197266, Val Loss: 1.8283430337905884, Val Accuracy: 38.10000228881836
Iteration 400, Epoch 1, Loss: 1.9263900518417358, Accuracy: 35.92970657348633, Val Loss: 1.7355294227600098, Val Accuracy: 43.099998474121094
Iteration 500, Epoch 1, Loss: 1.8832906484603882, Accuracy: 37.0914421081543, Val Loss: 1.64544677734375, Val Accuracy: 43.900001525878906
Iteration 600, Epoch 1, Loss: 1.8533260822296143, Accuracy: 38.0667610168457, Val Loss: 1.6734694242477417, Val Accuracy: 41.70000076293945
Iteration 700, Epoch 1, Loss: 1.8273297548

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [ ]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)

        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer=initializer)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer=initializer)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2))
        self.dropout1 = tf.keras.layers.Dropout(0.25)

        self.conv3 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer)
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.conv4 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer)
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2))
        self.dropout2 = tf.keras.layers.Dropout(0.25)

        self.conv5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer=initializer)
        self.bn5 = tf.keras.layers.BatchNormalization()
        self.pool3 = tf.keras.layers.MaxPooling2D((2, 2))
        self.dropout3 = tf.keras.layers.Dropout(0.25)

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(256, kernel_initializer=initializer)
        self.bn6 = tf.keras.layers.BatchNormalization()
        self.dropout4 = tf.keras.layers.Dropout(0.5)
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = tf.nn.relu(self.bn1(self.conv1(input_tensor), training=training))
        x = tf.nn.relu(self.bn2(self.conv2(x), training=training))
        x = self.pool1(x)
        x = self.dropout1(x, training=training)

        x = tf.nn.relu(self.bn3(self.conv3(x), training=training))
        x = tf.nn.relu(self.bn4(self.conv4(x), training=training))
        x = self.pool2(x)
        x = self.dropout2(x, training=training)

        x = tf.nn.relu(self.bn5(self.conv5(x), training=training))
        x = self.pool3(x)
        x = self.dropout3(x, training=training)

        x = self.flatten(x)
        x = tf.nn.relu(self.bn6(self.fc1(x), training=training))
        x = self.dropout4(x, training=training)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 3.009824275970459, Accuracy: 10.9375, Val Loss: 4.6044182777404785, Val Accuracy: 11.90000057220459
Iteration 700, Epoch 1, Loss: 1.5600272417068481, Accuracy: 44.59477615356445, Val Loss: 1.305759072303772, Val Accuracy: 54.400001525878906
Iteration 1400, Epoch 2, Loss: 1.0956251621246338, Accuracy: 60.809547424316406, Val Loss: 0.8900360465049744, Val Accuracy: 68.5
Iteration 2100, Epoch 3, Loss: 0.9242270588874817, Accuracy: 67.27262878417969, Val Loss: 0.8121842741966248, Val Accuracy: 71.30000305175781
Iteration 2800, Epoch 4, Loss: 0.8283480405807495, Accuracy: 70.73185729980469, Val Loss: 0.8028408288955688, Val Accuracy: 70.9000015258789
Iteration 3500, Epoch 5, Loss: 0.7621339559555054, Accuracy: 73.22654724121094, Val Loss: 0.839876115322113, Val Accuracy: 71.0
Iteration 4200, Epoch 6, Loss: 0.7151309847831726, Accuracy: 74.69255065917969, Val Loss: 0.715745747089386, Val Accuracy: 75.70000457763672
Iteration 4900, Epoch 7, Loss: 0.6553837060928345

Опишите все эксперименты, результаты. Сделайте выводы.

Простые полносвязные сети (FC) с SGD достигают максимум 45.7%, а добавление свёрточных слоёв даёт +8.9% за одну эпоху. Финальная CustomConvNet с BatchNorm и Dropout достигает 81.4% — это +35.7% против базовой модели.

LR=5e-4 замедляет сходимость на 5%, LR=1e-3 оптимален. SGD с большим LR (1e-2) даёт только 45.7%, тогда как Adam с 1e-3 — 54.4% за одну эпоху.

BatchNormalization + Dropout дают +26.7%

SGD с Momentum лучше чистого SGD: ThreeLayerConvNet (SGD+Momentum, lr=3e-3) показал 54.6% против 49.5% (чистый SGD, lr=5e-4).

Adam оптимален для глубоких сетей: Комбинация Adam + BatchNorm позволила достичь плавной сходимости без переобучения.